In [1]:

from qiskit.algorithms import VQE
from qiskit_nature.algorithms import (GroundStateEigensolver,
                                      NumPyMinimumEigensolverFactory)
from qiskit_nature.drivers import Molecule, UnitsType
from qiskit_nature.drivers.second_quantization import PySCFDriver, ElectronicStructureMoleculeDriver, ElectronicStructureDriverType
from qiskit_nature.transformers.second_quantization.electronic import FreezeCoreTransformer
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import ParityMapper

import matplotlib.pyplot as plt
import numpy as np
from qiskit_nature.circuit.library import UCCSD, HartreeFock
from qiskit.circuit.library import EfficientSU2
from qiskit.algorithms.optimizers import COBYLA, SPSA, SLSQP
from qiskit.opflow import TwoQubitReduction
from qiskit import BasicAer, Aer
from qiskit.utils import QuantumInstance
from qiskit.utils.mitigation import CompleteMeasFitter
from qiskit.providers.aer.noise import NoiseModel
from qiskit import Aer, IBMQ
from qiskit.providers.aer import AerSimulator
from qiskit.tools.visualization import plot_histogram, plot_circuit_layout, plot_gate_map
import pickle

In [2]:
# Save IBM account
#IBMQ.save_account('')
IBMQ.load_account()
# Choose the provider (if you have more than one)
provider = IBMQ.get_provider(hub='ibm-q-research-2', group='uminho-1', project='main')

# Show the possible backends
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q-research-2', group='uminho-1', project='main')>,
 <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q-research-2', group='uminho-1', project='main')>,
 <IBMQBackend('ibmq_santiago') from IBMQ(hub='ibm-q-research-2', group='uminho-1', project='main')>,
 <IBMQBackend('ibmq_bogota') from IBMQ(hub='ibm-q-research-2', group='uminho-1', project='main')>,
 <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q-research-2', group='uminho-1', project='main')>,
 <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q-research-2', group='uminho-1', project='main')>,
 <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q-research-2', group='uminho-1', project='main')>,
 <IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q-research-2', group='uminho-1', project='main')>,
 <IBMQSimulator('simulator_mps') from IBMQ(hub='ibm-q-research-2', group='uminho-1', project='main')>,
 <IBMQSimulator('simulator_extended_stabilizer') from IBMQ(hub='ibm-q-research-2', gr

In [3]:


# List queue times
for backend in provider.backends():
    if backend.configuration().simulator:
        continue
    try:
        qtime = backend.status().pending_jobs
    except:
        qtime = 'None'
    print(backend.name(), "has a queue time of", qtime)

ibmq_armonk has a queue time of None
ibmq_santiago has a queue time of None
ibmq_bogota has a queue time of None
ibmq_lima has a queue time of 48
ibmq_belem has a queue time of 32
ibmq_quito has a queue time of 29
ibmq_jakarta has a queue time of 20
ibmq_manila has a queue time of 35
ibm_lagos has a queue time of 31
ibm_nairobi has a queue time of 85
ibm_perth has a queue time of 14
ibm_oslo has a queue time of 49


# Quantum Chemistry
## Finding the minimum energy of moleculas
### Class of Quantum Computation of University of Minho
### 2022/2023
#### Maria Gabriela Jordão Oliveira, pg50599
#### Miguel Caçador Peixoto, pg

In [4]:
# Choose the "perfect" simulator
simulation_backend = Aer.get_backend("statevector_simulator")

# Choose the real device
real_device = provider.get_backend('ibm_perth')

# Create a simulator from the real device
sim_device = AerSimulator.from_backend(real_device)

In [5]:
# Exact solver for comparison
def exact_solver(problem, converter):
    '''
    This function finds the exact solution for a given problem and converter.
    This is a pure classical solver.
    '''
    solver = NumPyMinimumEigensolverFactory()
    calc = GroundStateEigensolver(converter, solver)
    result = calc.solve(problem)
    return result

# BeH2
##### Expected energy: -15.561353 hartree
##### Expected bond length: 1.291 Angstrom


In [6]:
def get_qubit_op_be(dist):
    '''
    Args:
    dist : Distance between Be and H in Angstrom
    
    Note -> BeH2 is a linear molecula. So the distance between Be and each H is the same.
    
    Returns:
    qubit_op : Qubit operator
    num_particles : Number of particles
    num_spin_orbitals : Number of spin orbitals
    problem : Problem. Electronic structure problem with driver and transformers. Freeze core transformation is used.
    converter : Converter. Qubit converter with parity mapper and two qubit reduction.
    '''
    # Define Molecule
    molecule = Molecule(
        # Coordinates in Angstrom
        geometry=[
            ["Be", [0.0, 0.0, 0.0]],
            ["H", [dist, 0.0, 0.0]],
            ["H", [-dist, 0.0, 0.0]]
        ],
        multiplicity=1,  # = 2*spin + 1
        charge=0,
    )

    driver = ElectronicStructureMoleculeDriver(
        molecule=molecule,
        basis="sto3g",
        driver_type=ElectronicStructureDriverType.PYSCF)

    # Get properties
    properties = driver.run()
    num_particles = (properties
                        .get_property("ParticleNumber")
                        .num_particles)
    num_spin_orbitals = int(properties
                            .get_property("ParticleNumber")
                            .num_spin_orbitals)

    # Define Problem, Use freeze core approximation, remove orbitals.
    problem = ElectronicStructureProblem(
        driver,
        [FreezeCoreTransformer(freeze_core=True)])
                              # remove_orbitals=[-3,-2])])

    second_q_ops = problem.second_q_ops()  # Get 2nd Quant OP
    num_spin_orbitals = problem.num_spin_orbitals
    num_particles = problem.num_particles
    #print(second_q_ops)
    # Get Hamiltonian
    mapper = ParityMapper()  # Set Mapper
    hamiltonian = second_q_ops[0]  # Set Hamiltonian
    # Do two qubit reduction
    converter = QubitConverter(mapper,two_qubit_reduction=True)
    reducer = TwoQubitReduction(num_particles)
    qubit_op = converter.convert(hamiltonian)
    qubit_op = reducer.convert(qubit_op)

    return qubit_op, num_particles, num_spin_orbitals, problem, converter

In [7]:

distances = np.arange(0.5, 3.0, 0.2)
exact_energies = []
vqe_energies = []
optimizer = SLSQP(maxiter=5)

# pylint: disable=undefined-loop-variable
for dist in distances:
    (qubit_op, num_particles, num_spin_orbitals,
                             problem, converter) = get_qubit_op_be(dist)
    result = exact_solver(problem,converter)
    exact_energies.append(result.total_energies[0].real)
    init_state = HartreeFock(num_spin_orbitals, num_particles, converter)
    var_form = UCCSD(converter,
                     num_particles,
                     num_spin_orbitals,
                     initial_state=init_state)
    vqe = VQE(var_form, optimizer, quantum_instance=simulation_backend)
    vqe_calc = vqe.compute_minimum_eigenvalue(qubit_op)
    vqe_result = problem.interpret(vqe_calc).total_energies[0].real
    vqe_energies.append(vqe_result)
    print(f"Interatomic Distance: {np.round(dist, 2)}",
          f"VQE Result: {vqe_result:.5f}",
          f"Exact Energy: {exact_energies[-1]:.5f}")

print("All energies have been calculated")

Interatomic Distance: 0.5 VQE Result: -13.68766 Exact Energy: -13.68826
Interatomic Distance: 0.7 VQE Result: -14.87056 Exact Energy: -14.87098
Interatomic Distance: 0.9 VQE Result: -15.36353 Exact Energy: -15.36382
Interatomic Distance: 1.1 VQE Result: -15.54904 Exact Energy: -15.54932
Interatomic Distance: 1.3 VQE Result: -15.59436 Exact Energy: -15.59471
Interatomic Distance: 1.5 VQE Result: -15.57514 Exact Energy: -15.57571
Interatomic Distance: 1.7 VQE Result: -15.52771 Exact Energy: -15.52878
Interatomic Distance: 1.9 VQE Result: -15.47106 Exact Energy: -15.47312
Interatomic Distance: 2.1 VQE Result: -15.41590 Exact Energy: -15.42019
Interatomic Distance: 2.3 VQE Result: -15.36937 Exact Energy: -15.37788
Interatomic Distance: 2.5 VQE Result: -15.33714 Exact Energy: -15.35149
Interatomic Distance: 2.7 VQE Result: -15.32040 Exact Energy: -15.34021
Interatomic Distance: 2.9 VQE Result: -15.31509 Exact Energy: -15.33693
All energies have been calculated


In [ ]:
import pickle
with open('sim_beh2.pkl', 'wb') as f:
    pickle.dump(vqe_energies, f)